In [1]:
import numpy as np
import pandas as pd
import random
import re


from collections import Counter
from sklearn.metrics import accuracy_score
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score
from sklearn.metrics import accuracy_score
from IPython.display import Image as PImage
from subprocess import check_call
from PIL import Image, ImageDraw, ImageFont

from google.colab import drive
drive.mount('/content/drive',force_remount = True)


# Loading the data
#train = pd.read_csv('../input/train.csv')
train = pd.read_csv('/content/drive/MyDrive/data/train.csv')
test = pd.read_csv('/content/drive/MyDrive/data/test.csv')
test_survival = pd.read_csv('/content/drive/MyDrive/data/gender_submission.csv')

# test = pd.read_csv('../input/test.csv')

# Store our test passenger IDs for easy access
PassengerId = test['PassengerId']

# Showing overview of the train dataset
train.head(3)

Mounted at /content/drive


,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S


Preprocessing the data for Titanic dataset

In [2]:
# Copy original dataset in case we need it later when digging into interesting features
# WARNING: Beware of actually copying the dataframe instead of just referencing it
# "original_train = train" will create a reference to the train variable (changes in 'train' will apply to 'original_train')
original_train = train.copy() # Using 'copy()' allows to clone the dataset, creating a different object with the same values

# Feature engineering steps taken from Sina and Anisotropic, with minor changes to avoid warnings
full_data = [train, test]

# Feature that tells whether a passenger had a cabin on the Titanic
train['Has_Cabin'] = train["Cabin"].apply(lambda x: 0 if type(x) == float else 1)
test['Has_Cabin'] = test["Cabin"].apply(lambda x: 0 if type(x) == float else 1)

# Create new feature FamilySize as a combination of SibSp and Parch
for dataset in full_data:
    dataset['FamilySize'] = dataset['SibSp'] + dataset['Parch'] + 1
# Create new feature IsAlone from FamilySize
for dataset in full_data:
    dataset['IsAlone'] = 0
    dataset.loc[dataset['FamilySize'] == 1, 'IsAlone'] = 1
# Remove all NULLS in the Embarked column
for dataset in full_data:
    dataset['Embarked'] = dataset['Embarked'].fillna('S')
# Remove all NULLS in the Fare column
for dataset in full_data:
    dataset['Fare'] = dataset['Fare'].fillna(train['Fare'].median())

# Remove all NULLS in the Age column
for dataset in full_data:
    age_avg = dataset['Age'].mean()
    age_std = dataset['Age'].std()
    age_null_count = dataset['Age'].isnull().sum()
    age_null_random_list = np.random.randint(age_avg - age_std, age_avg + age_std, size=age_null_count)
    # Next line has been improved to avoid warning
    dataset.loc[np.isnan(dataset['Age']), 'Age'] = age_null_random_list
    dataset['Age'] = dataset['Age'].astype(int)

# Define function to extract titles from passenger names
def get_title(name):
    title_search = re.search(' ([A-Za-z]+)\.', name)
    # If the title exists, extract and return it.
    if title_search:
        return title_search.group(1)
    return ""

for dataset in full_data:
    dataset['Title'] = dataset['Name'].apply(get_title)
# Group all non-common titles into one single grouping "Rare"
for dataset in full_data:
    dataset['Title'] = dataset['Title'].replace(['Lady', 'Countess','Capt', 'Col','Don', 'Dr', 'Major', 'Rev', 'Sir', 'Jonkheer', 'Dona'], 'Rare')

    dataset['Title'] = dataset['Title'].replace('Mlle', 'Miss')
    dataset['Title'] = dataset['Title'].replace('Ms', 'Miss')
    dataset['Title'] = dataset['Title'].replace('Mme', 'Mrs')

for dataset in full_data:
    # Mapping Sex
    dataset['Sex'] = dataset['Sex'].map( {'female': 0, 'male': 1} ).astype(int)
    
    # Mapping titles
    title_mapping = {"Mr": 1, "Master": 2, "Mrs": 3, "Miss": 4, "Rare": 5}
    dataset['Title'] = dataset['Title'].map(title_mapping)
    dataset['Title'] = dataset['Title'].fillna(0)

    # Mapping Embarked
    dataset['Embarked'] = dataset['Embarked'].map( {'S': 0, 'C': 1, 'Q': 2} ).astype(int)
    
    # Mapping Fare
    dataset.loc[ dataset['Fare'] <= 7.91, 'Fare'] 						        = 0
    dataset.loc[(dataset['Fare'] > 7.91) & (dataset['Fare'] <= 14.454), 'Fare'] = 1
    dataset.loc[(dataset['Fare'] > 14.454) & (dataset['Fare'] <= 31), 'Fare']   = 2
    dataset.loc[ dataset['Fare'] > 31, 'Fare'] 							        = 3
    dataset['Fare'] = dataset['Fare'].astype(int)
    
    # Mapping Age
    dataset.loc[ dataset['Age'] <= 16, 'Age'] 					       = 0
    dataset.loc[(dataset['Age'] > 16) & (dataset['Age'] <= 32), 'Age'] = 1
    dataset.loc[(dataset['Age'] > 32) & (dataset['Age'] <= 48), 'Age'] = 2
    dataset.loc[(dataset['Age'] > 48) & (dataset['Age'] <= 64), 'Age'] = 3
    dataset.loc[ dataset['Age'] > 64, 'Age'] ;

In [3]:
# Feature selection: remove variables no longer containing relevant information
drop_elements = ['PassengerId', 'Name', 'Ticket', 'Cabin', 'SibSp']
train = train.drop(drop_elements, axis = 1)
test  = test.drop(drop_elements, axis = 1)

X_train,y_train,X_test,y_test data

In [4]:
X_train = train.iloc[:,1:].values #train data all features to predict if passanger survived or not
X_train = np.array(X_train)

y_train = train.iloc[:,0].values #train data "survived"
y_train = np.array(y_train)

X_test = test.iloc[:100,:].values#test data all features 
X_test = np.array(X_test)

y_test = test_survival.iloc[:100,-1].values#Y test data for "survived"
y_test = np.array(y_test)


Decision Trees

In [5]:
class DecisionTree:
    
    def __init__(self, max_depth=10, min_samples_split=5, min_samples_leaf=2,criterion="gini"):
        self.max_depth = max_depth
        self.min_samples_split = min_samples_split
        self.min_samples_leaf = min_samples_leaf
        self.criterion = criterion
        self.criterion_function = None

    def fit(self, X, y,sample_weight=None):
      self.classes = len(np.unique(y))
      self.tree = self.buildTree(X, y)


    # Calculate entropy
    def entropy(self, y):
        frequency_count = np.bincount(y)
        normalized_frequency_count = frequency_count / len(y)
        entropy = -np.sum([i * np.log2(i) for i in normalized_frequency_count if i > 0])
        return entropy
    def gini_impurity(self,y):
        _, counts = np.unique(y, return_counts=True)
        probs = counts / len(y)
        impurity = 1 - np.sum(np.square(probs))
        return impurity
    #Calculate gini
    def gini(self, y):
      samples = len(y)
      gini = 1.0 - sum((np.sum(y == i) / samples) ** 2 for i in range(self.classes))
      return gini
    

    #Calculate missclassification
    def missclassification(self, y):
        samples = len(y)
        missclassification = 1.0 - np.max([np.sum(y == i) / samples for i in range(self.classes)])
        return missclassification

    def predict(self, X):
      y_pred = []
      for inputs in X:
          output = self.predict_helper(inputs)
          #print(output,"output")
          y_pred.append(output)
      return y_pred


    def predict_helper(self, inputs):
        node = self.tree
        while 'predicted_class' not in node:
            if inputs[node['splitting_feature']] < node['threshold']:
                node = node['left']
            else:
                node = node['right']
        return node['predicted_class']

    def calculate_threshold_classes(self,index_b,X,y):
      thresholds = []
      classes = []
      for i in range(len(y)):
              min_index = np.argmin(X[:, index_b])
              thresholds.append(X[min_index, index_b])
              classes.append(y[min_index])
              X = np.delete(X, min_index, axis=0)
              y = np.delete(y, min_index)
      return thresholds,classes

    def calculate_th(self,X,y,idx):
      return zip(*sorted(zip(X[:, idx], y)))

    def bestSplit(self, X, y):
      samples = len(y)
      if self.criterion == "entropy":
        criterion_function = self.entropy
      if self.criterion == "gini":
        criterion_function = self.gini
      if self.criterion == "missclassification":
        criterion_function = self.missclassification
      best_criterion = criterion_function(y)
      best_index = None
      best_threshold = None
      
      for index_b in range(X.shape[1]):
          thresholds, classes = self.calculate_th(X,y,index_b)
          
          num_left = [0] * self.classes #initialize it to 0 
          num_right = [np.sum(y == c) for c in range(self.classes)]
          for i in range(1, samples):
              c = classes[i - 1]
              num_left[c] += 1
              num_right[c] -= 1
              if i < self.min_samples_leaf or samples - i < self.min_samples_leaf:
                  continue
              criterion_left = criterion_function(num_left)
              criterion_right = criterion_function(num_right)
              criterion = (i * criterion_left + (samples - i) * criterion_right) / samples
              #print("criterion1",criterion_left,criterion_right)
              if thresholds[i] == thresholds[i - 1]:
                  continue
              #updating the variables that store the current best split found so far
              if criterion < best_criterion:
                  best_criterion = criterion
                  best_index = index_b
                  best_threshold = (thresholds[i] + thresholds[i - 1]) / 2
      return best_index, best_threshold

  

    def buildTree(self, X, y, depth=0):
      num_samples_per_class = []#create an empty list
      for i in range(self.classes):
        count = np.sum(y == i)
        num_samples_per_class.append(count)
      predicted_class = np.argmax(num_samples_per_class)
      node = {'depth': depth, 'n_samples': len(y), 'predicted_class': predicted_class}
      # Split recursively until maximum depth is reached or no more splits are possible
      if depth < self.max_depth:
          index, threshold = self.bestSplit(X, y)
          if index is not None:
              left_index = X[:, index] < threshold
              X_left = X[left_index] 
              y_left = y[left_index]
              X_right = X[~left_index]
              y_right = y[~left_index]
              if len(y_left) > self.min_samples_leaf and len(y_right) > self.min_samples_leaf:
                  node['splitting_feature'] = index
                  node['threshold'] = threshold
                  node['left'] = self.buildTree(X_left, y_left, depth + 1)
                  node['right'] = self.buildTree(X_right, y_right, depth + 1)
      return node

In [6]:
model1 = DecisionTree(max_depth=4, min_samples_split=10, min_samples_leaf=2,criterion="gini")
model1.fit(X_train,y_train)

y_pred = model1.predict(X_test)

acc = accuracy_score(y_test, y_pred)
print("Accuracy Decision tree",acc*100)

Accuracy Decision tree 61.0


Random Forest

In [7]:
class RandomForest:
    
    def __init__(self, input_classifier, num_trees, min_features):
        self.input_classifier = input_classifier
        self.num_trees = num_trees
        self.min_features = min_features
        self.trees = []
        self.index_features = None
        

    def fit(self, X, y):

        for i in range(self.num_trees):
            sample = len(X)
            # Sample with replacement
            index = np.random.choice(sample, sample, replace=True)
            X_sample = X[index]
            y_sample = y[index]
            # Select a random subset of features
            index_features = np.random.choice(X.shape[1], self.min_features, replace=False)
            X_sample = X_sample[:, index_features]
            tree = self.input_classifier()
            tree.fit(X_sample, y_sample)
            self.trees.append(tree)

    def predict(self, X):
        predictions = []
        for inputs in X:
            trees_predictions = []
            for tree in self.trees:
                # Make a prediction for each tree
                index_features = np.random.choice(X.shape[1], self.min_features, replace=False)
                tree_prediction = tree.predict_helper(inputs[self.index_features])
                trees_predictions.append(tree_prediction)
            # Select the majority vote from all the tree predictions
            majority_vote = Counter(trees_predictions).most_common(X.shape[0])[0][0]
            predictions.append(majority_vote)
        return predictions


In [8]:
model2 = RandomForest(input_classifier=DecisionTree,num_trees=10, min_features=1)
model2.fit(X_train, y_train)
y_pred = model2.predict(X_test)

acc = accuracy_score(y_test, y_pred)
print("Accuracy Random Forest", acc*100)

Accuracy Random Forest 61.0


AdaBoost

In [9]:
class AdaBoost:
    def __init__(self, weak_learner, num_learners, learning_rate):
        self.weak_learner = weak_learner
        self.num_learners = num_learners
        self.learning_rate = learning_rate

    def fit(self, X, y):
        # Initialize weights
        weights = np.full(len(X), 1/len(X))
        self.learners = []
        self.weights = []

        for i in range(self.num_learners):
            # Fit weak learner on weighted samples
            learner = self.weak_learner()
            learner.fit(X, y, sample_weight=weights)
            self.learners.append(learner)

            # Compute weighted error and update weights
            y_pred = learner.predict(X)
            error = np.sum(weights * (y_pred != y))
            alpha = self.learning_rate * np.log((1 - error) / error)
            weights = weights * np.exp(alpha * (y_pred != y))
            weights = weights / np.sum(weights)

            # Save weights for prediction
            self.weights.append(alpha)

            # Stop early if perfect fit achieved
            if error == 0:
                break
    def predict(self, X):
        predictions = np.ones(len(X))
        for i in range(len(self.learners)):
            learner = self.learners[i]
            alpha = self.weights[i]
            predictions += alpha * np.array(learner.predict(X))
        return np.sign(predictions)



In [10]:

model4 = AdaBoost(weak_learner = DecisionTree, num_learners = 50, learning_rate = 0.1)
model4.fit(X_train,y_train)
y_pred_Boosting = model4.predict(y_test)
acc_Boosting = accuracy_score(y_test, y_pred_Boosting)
print("Accuracy Boosting :", acc_Boosting*100)

Accuracy Boosting : 39.0
